In [1]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import ast

# TODO: there must be some cleaner way use local packages with poetry, but lets use this quick hack for now...
import sys
sys.path.append('../../')  # Adjust the path as needed
from utils.spacy_visualizer import SpacyLabelVisualizer


In [2]:
load_dotenv(find_dotenv())
OPENAI_SECRET_KEY = os.getenv('OPENAI_SECRET_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_SECRET_KEY

In [3]:
prompt1 = open('system_prompts/prompt_2.txt', 'r').read()

In [4]:
class ActFrameConstructor:
  def __init__(self, prompt: str, prompt_id: str):
    self._seed = 42 # ensure reproducable results
    self._temperature = 0.0 # ensure deterministic results
    self._client = OpenAI() 
    self._model = "gpt-3.5-turbo-1106" 
    self._prompt_id = prompt_id
    # self._model="gpt-4-1106-preview", # this is expensive
    self.system_prompt = prompt   

    
  def id(self):
    return f"model:{self._model}:seed:{self._seed}:temperature:{self._temperature}:prompt:{self._prompt_id}"
    
  def request_full_response(self, input: str) -> str:
    response = self._computeReponse(input)
    message = self._extract_message_from_response(response)
    return message
        
        
  # MARK: - Private Methods
  def _computeReponse(self, sentence):
     response = self._client.chat.completions.create(
      seed=self._seed, 
      temperature=self._temperature, 
      model=self._model,
      # response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": self.system_prompt},
        {"role": "user", "content": sentence},
      ],
      # stream=True,
    )
     return response
  
     
  def _extract_message_from_response(self, response) -> str:
    return response.choices[0].message.content
  
  def _extract_final_line(self, message: str) -> str:
    message.split("```\n")[-1]
    # remove trailing \n```
    message = message.split("```\n")[-1].replace("\n```", "")
    return message
    
    



In [5]:
predictor = ActFrameConstructor(prompt=prompt1, prompt_id="prompt_v1")

In [6]:
# predictor.request_full_response("I want to go to the movies")

In [7]:
article_name = "GDPR_art6"
article = open(f'data/{article_name}.txt', 'r').read()

In [8]:
answer = predictor.request_full_response(article)

In [10]:
# save answer in a file
answer_id  = predictor.id() + f":article:{article_name}" + f":date:{pd.Timestamp.now()}"
with open(f'answers/{answer_id}.md', 'w') as f: # write as markdown; as the reponse will contain a json block this will allow for an easier read.
    f.write(answer)

In [ ]:
# TODO: use another pormpt with json mode and extract answer
# TODO: give room to think (COT); given the answers right now, it seems to jump straight to the answer instead.